# Using Pandas together with SQL

In [ ]:
# Install the SQLAlchemy library if it is not installed
# !sudo -H pip3 install -U sqlalchemy pandas

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Make the graphs a bit prettier, and bigger
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 7)

## Importing SQL results into DataFrames using read_sql



The `read_sql` function of Pandas allows us to create a dataframe directly from a SQL query. To execute the query, we first setup the connection to the database using the SQLAlchemy library.

In [ ]:
from sqlalchemy import create_engine

In [ ]:
conn_string_imdb = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='student', 
    password='dwdstudent2015', 
    host = 'db.ipeirotis.org', 
    port=3306, 
    db='imdb',
    encoding = 'utf-8'
)
engine_imdb = create_engine(conn_string_imdb)

Let's start with a simple example. We issue an SQL query, and get back the results loaded in a dataframe.

In [ ]:
query = '''
SELECT * FROM actors LIMIT 10
'''

In [ ]:
df_actors = pd.read_sql(query, con=engine_imdb)

In [ ]:
df_actors

In [ ]:
len(df_actors)

## Aggregation Calculations: Pandas or SQL? 



Now let's work on a slightly more advanced example. We want to analyze the number of movies over time.

### Basic Option: Fetch all data, analyze in Pandas

Let's do the simple thing first. We will fetch all the data from the movies table and then do a pivot table on top. Since we care about efficiency, we will also time the operation.

In [ ]:
%%time
query = '''SELECT * FROM movies'''
df_basic = pd.read_sql(query, con=engine_imdb)

In [ ]:
len(df_basic)

So, notice that it takes 2-3 seconds to fetch the data from SQL and create the dataframe, as we need to fetch almost 400K records. 

Once we have the records, we can then compute a pivot table:

In [ ]:
%%time
# Counting movie IDs returns all the movies within the year
# Counting movie ranks returns all the movies that have 
# a non-empty "rank" value (i.e., they have been rated)
pivot = df_basic.pivot_table(
    index = 'year',
    aggfunc = 'count',
    values = ['id', 'rank']
)
# Rename the columns
pivot.columns = ['all_movies', 'rated_movies']

In [ ]:
# And let's check a few lines of the table
pivot.sample(5)

And we can then plot the results:

In [ ]:
pivot.plot()

### Better option: Aggregation in SQL, fetch only necessary data

Now let's push the computation on the SQL server instead, using a GROUP BY and COUNT aggregates in SQL.

In [ ]:
%%time
query = '''
SELECT year, COUNT(*) AS all_movies, COUNT(rank) AS rated_movies
FROM movies 
GROUP BY year
ORDER BY year
'''
df_movies = pd.read_sql(query, con=engine_imdb)

In [ ]:
len(df_movies)

In [ ]:
df_movies.sample(5)

Notice that the same calculation was done in a few (4-5) **milliseconds**. The SQL query that we used earlier it took **seconds** to execute. In fact, the **pivot** table calculation, executed after fetching all the data took longer than executing the GROUPBY/COUNT SQL query and fetching the results.

While in this example the difference is negligible, once you deal with datasets that have millions, or tens of millions of rows, the savings become material and significant.

#### Plotting: The importance of index

Let's try to plot the results. In pandas, the simple `plot()` command will use the index as the x-axis, and will plot all the numeric columns, as a line plot.

In [ ]:
# The plot() command takes the index (the first "column") of the dataframe
# and makes that the x-axis.
# Then it plots *ALL* the numeric columns as a line
df_movies.plot()

We do not want to plot the `year` variable as a line. So, we select just the other two columns and plot.

In [ ]:
# First step: We can eliminate the "year" line by selecting 
# the columns that we want to plot
# To select columns, we pass a list of the column names that
# we want to keep in square brackets
df_movies[ ["all_movies", "rated_movies"] ].plot() 
# still the x-axis does not list the year

A bit better. `year` is not appearing anymore, but we still do not have `year` as the x-axis. 

To make `year` the x-axis, we need to make it the index of the dataframe:

In [ ]:
df_movies_2 = df_movies.set_index('year')
df_movies_2.sample(5)

Now the plot has the year as the x-axis, and the labels are proper.

In [ ]:
df_movies_2.plot()

#### (Optional, but useful) Changing data types: Int vs Datetime

In our index above, the "year" variable is an integer:

In [ ]:
df_movies_2.index.dtype

This is mostly fine, but we can leverage the time series processing capabilities of Pandas by converting `year` to a date.

In [ ]:
# We first convert the index into datetime.
df_movies_2.index = pd.to_datetime(df_movies_2.index, format='%Y')

In [ ]:
df_movies_2.sample(5)

Now we can do the `resample` the dates in the index. For example, we can compute numbers over decades:

In [ ]:
df_movies_2.resample('10Y').sum()

## Exercise

* Connect to the Facebook database, and use the `MemberSince` variable from the `Profiles` table to plot the growth of Facebook users. Use the following information:
>    user='student', 
>    password='dwdstudent2015', 
>    host = 'db.ipeirotis.org', 
>    port=3306, 
>    db='facebook'
* (_Learn something new_) Use the [cumsum()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.cumsum.html) function of Pandas and plot the total number of registered users over time.

### Solution

In [ ]:
conn_string_fb = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='student', 
    password='dwdstudent2015', 
    host = 'db.ipeirotis.org', 
    port=3306, 
    db='facebook',
    encoding = 'utf-8'
)
engine_fb = create_engine(conn_string_fb)

In [ ]:
%%time
# Naive approach, fetch all the data first
query = 'SELECT * FROM Profiles'
df = pd.read_sql(query, con=engine_fb)
pivot = df.pivot_table(
    index='MemberSince',
    values='ProfileID',
    aggfunc='count'    
)
# Calculate weekly signups
weekly_signups = pivot.resample('1W').sum()

In [ ]:
%%time
# Push calculations into SQL
query = 'SELECT MemberSince, COUNT(ProfileID) as signups FROM Profiles GROUP BY MemberSince'
df = pd.read_sql(query, con=engine_fb)
df.set_index("MemberSince", inplace=True)
weekly_signups = df.resample('1W').sum()

In [ ]:
weekly_signups.plot()

In [ ]:
df.cumsum().plot()

### Further Examples with SQL and Pandas

Now let's run a query to get the political views of Facebook users, broken down by gender.

In [ ]:
conn_string_fb = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
    user='student',
    password='dwdstudent2015',
    host='db.ipeirotis.org',
    port=3306,
    db='facebook')
engine_fb = create_engine(conn_string_fb)

In [ ]:
polviews_by_gender = '''
SELECT Sex, PoliticalViews, COUNT(*) AS cnt 
FROM Profiles 
WHERE Sex IS NOT NULL AND PoliticalViews IS NOT NULL 
GROUP BY Sex, PoliticalViews
ORDER BY  PoliticalViews, Sex
'''

And let's get the dataframe:

In [ ]:
df = pd.read_sql(polviews_by_gender, con=engine_fb)
df

In [ ]:
df.dtypes

We will now convert the PoliticalViews column into an **ordered Categorical variable**. This is not strictly necessary, but it will be useful later.
 It ensures that Political Views appear in an order according to their political spectrum, as opposed to alphabetical.

In [ ]:
df.PoliticalViews = pd.Categorical(df.PoliticalViews,
    categories = ['Very Liberal', 'Liberal', 'Moderate', 'Conservative', 'Very Conservative', 'Libertarian', 'Apathetic', 'Other'], 
    ordered=True
)

In [ ]:
# Let's plot this!
# Bleh, this is really fugly...
# Remember that the index of the dataframe becomes the default x-axis
df.plot(kind='bar')

In [ ]:
# Pivot, baby!
# Now the index contains the Political Views, which will be our x-axis
dfp = pd.pivot_table(data = df, index='PoliticalViews', columns='Sex', values='cnt', aggfunc='sum')
dfp

In [ ]:
dfp.plot(kind='bar')

#### Normalizing values in Pandas

Now, let's see a bit how we can normalize the values in Pandas, by performing operations on the columns and rows.

In [ ]:
dfp

In [ ]:
dfp.sum() # sums across the rows

In [ ]:
dfp.T.sum()

In [ ]:
dfp.sum(axis='index') # sums across the rows (equivalent to dfp.sum() and dfp.sum(axis=0) )

In [ ]:
dfp.sum(axis='columns') # this one sums across the columns (axis=1)

In [ ]:
dfp.sum()

In [ ]:
# Let's normalize the columns, as we have more females than males, and it seems that there are always more women
dfp_norm = dfp / dfp.sum()
dfp_norm

In [ ]:
dfp_norm.plot(kind='bar')

In [ ]:
# Instead of dfp / dfp.sum(), we can also use the .div() method, for dividing the entries with the sum()
# Note that, by definition, the dfp / dfp.sum() operation divides  column-wise, not row-wise.
dfp_norm = dfp.div( dfp.sum(), axis='columns' )
dfp_norm

In [ ]:
dfp

In [ ]:
# And now let's do the division by normalizing the values row-wise,
# to find the fraction of males/females within each political category
dfp_norm2 = dfp.div( dfp.sum(axis='columns'), axis='index' ).sort_index()
dfp_norm2

In [ ]:
dfp_norm2.plot(kind='barh')

In [ ]:
dfp_norm2.plot(kind='bar', rot=45)

#### Exercise

a. Use the tables `RelationshipStatus` and `LookingFor`, and show create a plot with a breakdown of what people in different relationship statuses are looking for. To make things more readable (and to practice a bit SQL), remove from the output all combinations that have less than 10 students in them. The plot can use the absolute counts.

b. Normalize the results and plot again. To get experience with normalization, try to normalize both by Status (eg "80% of the people who are in a relationship are looking for Friendship") and by Relationship Status (eg "70% of the people who are looking for Random Play are Single"). 

In [ ]:
query = '''
SELECT R.Status, L.LookingFor, COUNT(*) AS cnt
FROM Relationship R INNER JOIN LookingFor L ON R.ProfileID = L.ProfileID
GROUP BY R.Status, L.LookingFor
HAVING cnt>10
'''
df = pd.read_sql(query, con=engine_fb)
df

In [ ]:
pivot = df.pivot_table(
    index='Status',
    columns='LookingFor',
    values='cnt'
)
pivot

In [ ]:
# Raw numbers, by relationship status
pivot.plot(kind='barh', figsize=(10,5))

In [ ]:
# Raw numbers, by looking for (taking the transpose)
pivot.T.plot(kind='barh', figsize=(10,5))

In [ ]:
# Every column (LookingFor) sums up to 1
pivot / pivot.sum()

In [ ]:
# Every column (LookingFor) sums up to 1
(pivot / pivot.sum()).T

In [ ]:
(pivot / pivot.sum()).T.plot(kind='barh')

In [ ]:
# Every column (Relationship Status) sums up to 1
pivot.T / pivot.T.sum()

### Facebook, Favorite Books, and Political views

Now let's do an analysis that examines book preferences and how they correlated with political leanings.

We will start by fetching the favorite books for students that declared themselves as Liberal or Conservative.

In [ ]:
books = '''
SELECT B.Book, P.PoliticalViews, COUNT(*) AS cnt 
FROM Profiles P JOIN FavoriteBooks B ON B.ProfileID = P.ProfileId  
WHERE PoliticalViews IS NOT NULL AND B.Book IS NOT NULL 
      AND (PoliticalViews = 'Liberal' OR PoliticalViews = 'Conservative')
AND B.Book IN (
    SELECT Book 
    FROM FavoriteBooks B JOIN Profiles P ON B.ProfileID = P.ProfileId  
    WHERE (P.PoliticalViews = 'Liberal' OR P.PoliticalViews = 'Conservative')
    GROUP BY Book HAVING COUNT(DISTINCT P.ProfileID)>10
)
GROUP BY B.Book, P.PoliticalViews;
'''

In [ ]:
df_books = pd.read_sql(books, con=engine_fb)
df_books.head(10)

In [ ]:
len(df_books)

In [ ]:
dfp = df_books.pivot_table(
    index='Book', 
    columns='PoliticalViews', 
    values='cnt')
dfp.head(10)

Notice the `NaN` values for the entries where we had no users falling into that group. Since we will want to do calculations for these books as well, we will use the `fillna` command to fill these entries with a default value (in our case, 0)

In [ ]:
# Fill the NaN entries with the value 0 
dfp = df_books.pivot_table(
    index='Book', 
    columns='PoliticalViews', 
    values='cnt').fillna(0)
dfp.head(10)

**Normalization**: We now want to normalize the entries before proceeding further. Let's take a look at the breakdown of political views in the database.

In [ ]:
polviews = '''
SELECT PoliticalViews, COUNT(*) AS cnt 
FROM facebook.Profiles
GROUP BY PoliticalViews
'''

In [ ]:
df_polviews = pd.read_sql(polviews, con=engine_fb)
df_polviews

Since we have many more conservatives than liberals, let's create a new column that calculates the **percentage** of liberal and conservative students that liked each book. For simplicity, we just enter directly the values 6461 (number of liberals) and 936 (number of conservatives). We add the `+1` in the numerator to avoid division by zero later on. _As practice, try to fetch the values 936 and 6461 directly from the database, and automate the calculation._

In [ ]:
dfp["Liberal_perc"] = (dfp["Liberal"] +1)  / 6461
dfp["Conservative_perc"] = (dfp["Conservative"] +1)  / 936

In [ ]:
dfp

#### Lift

Now that we have the normalized values, we can compute the **lift** for each book. The lift is the ratio between the percentage of liberals and the percentage of convervatives. A book with `lift==1` will be equally read by both conservatives and liberals. Books that have lifts significantly higher or lower than 1, reveal preferences to be read by one side of the political spectrum.

In [ ]:
dfp["lift_liberal"] = dfp["Liberal_perc"] / dfp["Conservative_perc"]
dfp["lift_conservative"] = dfp["Conservative_perc"]  / dfp["Liberal_perc"]

In [ ]:
dfp

#### Log-odds

One common tranformation is to take the `log` of the lift. We call this metric **log odds**. In that case, the `lift==1` corresponds to a `log_odds` of 0. Negative values indicate negative association, and positive values indicate positive association. A nice property of log-odds is that they are **additive**, which means that summing up log-odds makes (mathematical) sense, under some reasonably general conditions. (The details are beyond the scope of this course, but you can learn more in the data mining class.)

In [ ]:
import numpy as np
dfp["log_odds_liberal"]      =  np.log(dfp["lift_liberal"])
dfp["log_odds_conservative"] =  np.log(dfp["lift_conservative"])

In [ ]:
show_columns = ["lift_liberal", "log_odds_liberal", "lift_conservative", "log_odds_conservative", "Liberal", "Conservative"]

In [ ]:
liberal_books = (dfp[show_columns]
                 .sort_values("lift_liberal", ascending=False)
                 .head(10)
                )
liberal_books

In [ ]:
conservative_books = (dfp[show_columns]
                      .sort_values("lift_conservative", ascending=False)
                      .head(10)
                     )
conservative_books

In [ ]:
plot = conservative_books.lift_conservative.plot(kind='barh', figsize=(15,5))
plot.set_xlabel("Lift for Conservatives")

In [ ]:
plot = liberal_books.lift_liberal.plot(kind='barh', figsize=(15,5))
plot.set_xlabel("Lift for Liberals")

#### Exercise

We have seen how to compute the log-odds between liberal-conservative for each book. Given this information, we can try to estimate political leanings of students. You can do this by summing the log-odds of their favorite books. 

Steps: 
1. Create a table with the log-odds of the books. 
2. Join the table with the log-odds with the book preferences table.
3. Sum the log-odds score for each student.

Evaluation:
* You have students that have declared their political preferences as Liberal, Conservative, Very Liberal, Very Conservative. Examine the scores for these students, to check how well this technique works. The simplest way is to compute the average (mean) log-odds for students that fall into the different groups. Alternatively, you can try to plot the full distribution of scores.
* Calculate a score for each student that did not declare a political view but has listed Favorite Books.

Notes: 
* You can do the work in MySQL or in Pandas. If you decide to work purely in Pandas, the [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html#pandas.DataFrame.merge) command allows you to perform joins between dataframes, in way similar to SQL.